In [7]:
ip="localhost"

In [27]:
! curl -X GET localhost:9200/_cat/indices?v

health status index  uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   finger FNCEOVgCQLORR1BCW6huAA   1   1          0            0       208b           208b


In [5]:
! curl -X GET localhost:9200/right_thumb_fem/_doc/1

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [right_thumb_fem]","resource.type":"index_expression","resource.id":"right_thumb_fem","index_uuid":"_na_","index":"right_thumb_fem"}],"type":"index_not_found_exception","reason":"no such index [right_thumb_fem]","resource.type":"index_expression","resource.id":"right_thumb_fem","index_uuid":"_na_","index":"right_thumb_fem"},"status":404}

In [11]:
! curl -X GET localhost:9200/right_thumb_fem

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [right_thumb_fem]","resource.type":"index_or_alias","resource.id":"right_thumb_fem","index_uuid":"_na_","index":"right_thumb_fem"}],"type":"index_not_found_exception","reason":"no such index [right_thumb_fem]","resource.type":"index_or_alias","resource.id":"right_thumb_fem","index_uuid":"_na_","index":"right_thumb_fem"},"status":404}

In [28]:
! curl -X DELETE localhost:9200/*/

{"acknowledged":true}

In [8]:
def create_index():
    global ip
    from elasticsearch import Elasticsearch
    es = Elasticsearch(hosts=ip)
    request_body = {
        "settings" : {
            "number_of_shards": 1,
            "number_of_replicas": 1,

                "similarity": {
                  "scripted_tfidf": {
                    "type": "scripted",
                    "script": {
                      "source": "double t_idf = Math.ceil(Math.log((field.docCount+1.0)/(term.docFreq+1.0))); return doc.freq*t_idf;"
                    }
                  }
                }
              },
        "mappings": {
                "properties": {
                    "minutia": {"type":"nested", 
                                "properties":{
                                    "text": {"type":"text", "similarity":"scripted_tfidf"},
                                    "min_id":{"type":"text"}
                                    
                                }},
                   "id" : {"type":"text"},
                }
            
        }
    }

    es.indices.create(index = 'finger1', body = request_body,request_timeout=10000)
    return 1



create_index()
#hashs.map(lambda x:create_index(x)).collect()
print('OK')

OK


In [12]:
def create_index():
    global ip
    from elasticsearch import Elasticsearch
    es = Elasticsearch(hosts=ip)
    request_body = {
        "settings" : {
            "number_of_shards": 1,
            "number_of_replicas": 1,

                "similarity": {
                  "scripted_tfidf": {
                    "type": "scripted",
                    "script": {
                      "source": "double tf = Math.sqrt(doc.freq); double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; double norm = 1/Math.sqrt(doc.length); return query.boost * tf * idf * norm;"
                    }
                  }
                }
              },
        "mappings": {
                "properties": {
                    "minutia": {"type":"nested", 
                                "properties":{
                                    "text": {"type":"text", "similarity":"scripted_tfidf"},
                                    "min_id":{"type":"text"}
                                    
                                }},
                   "id" : {"type":"text"},
                }
            
        }
    }

    es.indices.create(index = 'finger1', body = request_body,request_timeout=10000)
    return 1



create_index()
#hashs.map(lambda x:create_index(x)).collect()
print('OK')

OK


In [8]:
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch(hosts=ip)

In [9]:
es.indices.delete(index='finger')

{'acknowledged': True}

In [4]:
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch(hosts=ip)

a = { "_op_type":"create",
                "_index": "finger1",
                "_source": {
                    "id": 1,
                    "minutia": [
                        {"min_id":1,
                        "text": "0_1 1_2 2_1 3_1 4_1 6_1"
                    },
                        {"min_id":2,
                        "text": "0_2 4_3 4_2 4_1 1_2 3_1 6_1"}
                    ]
                }
        } 


b = { "_op_type":"create",
                "_index": "finger1",
                "_source": {
                    "id": 2,
                    "minutia": [
                        {"min_id":1,
                        "text": "0_3 1_4 2_1 3_2 4_1 6_1"
                    },
                        {"min_id":2,
                        "text": "0_2 4_3 4_1 4_4 1_5 3_6 6_1"},
                    ]
                }
        } 

In [5]:
helpers.bulk(es, [a,b],request_timeout=1000)


(2, [])

In [23]:
query1 ={"_source":["_id"],"from":0,"size":100,
  "query": {
        "nested": {
          "path": "minutia",
          "query": {
            "bool": {
              "should": [
                { "match":{"minutia.text": "0_3 1_4 2_1 3_2 4_1 6_1" }},

              ]
            }
          },
        "score_mode": "max"
        }
      }
    }

In [ ]:
es.search(body=query1, index='finger1')

In [23]:
def a():
    for text in ["aa",["bb"]]:
        yield {"index": "finger"}
        yield {"_source": ["id"], "from": 0, "size": 100,
                                   "query": {
                                       "nested": {
                                           "path": "minutia",
                                           "query": {
                                               "bool": {
                                                   "should": [
                                                       {"match": {"minutia.text": text }},

                                                   ]
                                               }
                                           },
                                           "score_mode": "max"
                                       }
                                   }
                                   }

In [24]:
list(a())

[{'index': 'finger'},
 {'_source': ['id'],
  'from': 0,
  'size': 100,
  'query': {'nested': {'path': 'minutia',
    'query': {'bool': {'should': [{'match': {'minutia.text': 'aa'}}]}},
    'score_mode': 'max'}}},
 {'index': 'finger'},
 {'_source': ['id'],
  'from': 0,
  'size': 100,
  'query': {'nested': {'path': 'minutia',
    'query': {'bool': {'should': [{'match': {'minutia.text': ['bb']}}]}},
    'score_mode': 'max'}}}]

In [6]:
query1 =[{"index":"finger1"},{"_source":["id"],"from":0,"size":100,
  "query": {
        "nested": {
          "path": "minutia",
          "query": {
            "bool": {
              "should": [
                { "match":{"minutia.text": "0_3 1_4 2_1 3_2 4_1 6_1" }},

              ]
            }
          },
        "score_mode": "max"
        }
      }
    }, {"index":"finger1"}, {"_source":["_id"],"from":0,"size":100,
  "query": {
        "nested": {
          "path": "minutia",
          "query": {
            "bool": {
              "should": [
                { "match":{"minutia.text": "0_3 1_4 2_1 3_2 4_1 6_1" }},

              ]
            }
          },
        "score_mode": "max"
        }
      }
    }]

In [7]:
es.msearch(body=query1)

{'took': 790,
 'responses': [{'took': 789,
   'timed_out': False,
   '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
   'hits': {'total': {'value': 2, 'relation': 'eq'},
    'max_score': 4.0,
    'hits': [{'_index': 'finger1',
      '_type': '_doc',
      '_id': '0NdZ-nUBPvGKT-6Mell6',
      '_score': 4.0,
      '_source': {'id': 2}},
     {'_index': 'finger1',
      '_type': '_doc',
      '_id': 'z9dZ-nUBPvGKT-6Mell6',
      '_score': 1.0,
      '_source': {'id': 1}}]},
   'status': 200},
  {'took': 786,
   'timed_out': False,
   '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
   'hits': {'total': {'value': 2, 'relation': 'eq'},
    'max_score': 4.0,
    'hits': [{'_index': 'finger1',
      '_type': '_doc',
      '_id': '0NdZ-nUBPvGKT-6Mell6',
      '_score': 4.0,
      '_source': {}},
     {'_index': 'finger1',
      '_type': '_doc',
      '_id': 'z9dZ-nUBPvGKT-6Mell6',
      '_score': 1.0,
      '_source': {}}]},
   'status': 200}]}

In [42]:
query1 ={"_source":["_id"],"from":0,"size":100,
  "query": {
        "nested": {
          "path": "minutia",
          "query": {
            "bool": {
              "should": [
                { "more_like_this":{"max_query_terms": 100,"min_term_freq":1, "fields": ["minutia.text"], "like": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 44_66 45_16 47_165 48_8 54_71 55_24 56_16 57_191 58_238 59_64 64_71 65_16 66_16 67_191 68_238 69_64 74_2 75_16 77_141 78_72 90_70 91_48 93_24 94_64 99_8 100_70 101_49 102_3 103_24 104_194 110_70 111_49 112_99 113_24 114_194 120_2 122_99 123_24 124_64 132_99 " }},

              ]
            }
          }
        }
      }
    }

In [43]:
es.search(body=query1, index='right_thumb_fem')

{'took': 51,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 100, 'relation': 'eq'},
  'max_score': 4.403784,
  'hits': [{'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '7',
    '_score': 4.403784,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '69',
    '_score': 4.329981,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '82',
    '_score': 4.3229046,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '10',
    '_score': 4.231391,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '3',
    '_score': 4.221138,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '96',
    '_score': 4.1909204,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '14',
    '_score': 4.1851,
    '_source': {}},
   {'_i

In [40]:
 query1 = {"_source":["_id"],"from":0,"size":100, "query": {"nested": { "path": "minutia", "query": { "bool": {"must": [ { "match" : { "minutia.text":"0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 44_66 45_16 47_165 48_8 54_71 55_24 56_16 57_191 58_238 59_64 64_71 65_16 66_16 67_191 68_238 69_64 74_2 75_16 77_141 78_72 90_70 91_48 93_24 94_64 99_8 100_70 101_49 102_3 103_24 104_194 110_70 111_49 112_99 113_24 114_194 120_2 122_99 123_24 124_64 132_99 " }}]}}}}}

In [41]:
es.search(body=query1, index='right_thumb_fem')

{'took': 77,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 100, 'relation': 'eq'},
  'max_score': 4.403784,
  'hits': [{'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '7',
    '_score': 4.403784,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '69',
    '_score': 4.329981,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '82',
    '_score': 4.3229046,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '3',
    '_score': 4.2372646,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '10',
    '_score': 4.231391,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '96',
    '_score': 4.1909204,
    '_source': {}},
   {'_index': 'right_thumb_fem',
    '_type': '_doc',
    '_id': '14',
    '_score': 4.1851,
    '_source': {}},
   {'_

In [37]:
 query1 = {"_source":["_id"],"from":0,"size":100, "query": {"nested": { "path": "minutia", "query": { "bool": {"must": [ { "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 115_2 116_49 125_70 126_49 127_136 135_70 136_49 137_136 145_2 146_49 161_128 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 53_8 62_1 82_136 139_66 140_16 148_8 149_99 150_24 158_8 159_99 160_24 169_66 170_16 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 54_70 55_16 64_199 65_48 66_128 74_70 75_16 147_33 148_140 149_64 157_49 158_140 159_64 163_8 166_4 167_49 168_140 174_64 176_142 177_113 186_142 187_113 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 49_8 50_128 59_8 60_196 70_64 82_136 91_134 101_134 111_4 115_2 116_49 125_70 126_49 127_136 135_70 136_49 137_136 145_2 146_49 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 46_132 55_17 56_140 57_32 65_17 66_140 67_96 76_140 77_32 102_32 112_97 115_16 122_97 124_6 125_49 126_128 132_32 134_6 135_49 136_132 144_2 145_48 146_128 165_16 174_6 175_51 176_177 185_127 186_177 187_136 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_142 37_113 46_142 47_113 56_4 57_32 67_1 77_33 78_140 79_64 82_32 87_33 88_140 89_64 92_33 93_8 94_64 97_33 98_140 99_64 102_33 103_156 104_64 112_33 113_28 114_64 122_1 123_8 124_64 147_33 148_32 154_2 155_16 156_4 157_119 158_235 159_16 164_70 165_48 166_132 167_255 168_251 169_16 174_70 175_48 176_132 177_173 178_99 179_16 185_16 187_140 188_98 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 55_70 56_49 59_8 60_196 65_70 66_49 69_8 70_198 71_160 75_70 76_49 79_8 80_198 81_32 102_32 112_97 122_96 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 86_4 87_99 88_8 96_4 97_99 98_136 100_4 106_4 107_239 108_106 110_198 111_32 116_1 117_140 118_98 119_24 120_198 121_33 127_140 128_98 130_198 131_32 157_140 158_66 166_17 167_140 168_98 175_2 176_49 177_140 178_98 185_70 186_49 187_136 188_64 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_4 37_49 38_136 46_134 47_49 48_136 53_8 55_17 56_206 57_33 62_3 63_136 65_17 66_206 67_32 72_1 73_8 76_132 77_32 97_4 104_66 107_140 108_98 113_8 114_198 115_16 116_17 117_140 118_98 123_8 124_198 125_16 126_17 127_140 128_98 133_8 134_66 136_17 137_140 138_98 147_140 148_64 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 49_2 60_16 74_2 81_128 84_70 85_16 91_198 94_199 95_48 96_128 100_16 101_198 104_70 105_16 109_2 110_57 111_196 119_2 120_57 121_196 129_2 130_16 131_128 151_128 158_8 160_19 161_192 167_3 168_24 169_66 170_16 177_35 178_28 179_194 187_1 188_24 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 55_70 56_49 60_68 65_70 66_49 70_70 71_32 75_70 76_49 81_32 92_33 102_115 103_136 112_115 113_136 122_33 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 91_1 109_8 110_196 116_134 117_168 118_1 119_24 120_198 121_32 126_223 127_188 128_99 129_24 130_196 131_32 136_215 137_188 138_98 139_8 140_132 146_132 147_140 148_64 167_4 168_64 177_140 178_98 187_140 188_98 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 109_66 118_8 119_99 120_16 122_1 123_8 124_64 128_8 129_99 130_16 132_3 133_24 134_192 139_66 142_3 143_24 144_192 152_1 153_8 154_64 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 37_33 38_140 44_99 45_24 47_33 48_140 49_96 53_8 54_99 55_24 56_128 57_1 58_140 59_64 64_67 65_16 79_66 80_64 88_8 89_111 90_214 91_32 98_9 99_239 100_246 101_32 108_9 109_127 110_230 111_32 118_1 119_24 120_198 121_32 129_8 130_196 145_16 154_6 155_48 156_128 164_70 165_49 166_128 174_2 175_48 176_128 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 59_8 60_196 69_8 70_198 71_32 79_8 80_198 81_32 95_68 101_132 104_8 105_198 106_32 110_16 111_206 114_8 115_198 116_32 120_16 121_206 123_2 124_24 125_198 126_32 131_132 133_103 134_16 135_128 143_231 144_48 145_128 153_198 154_48 155_128 163_6 164_48 166_33 174_48 175_4 176_99 177_8 185_4 186_115 187_8 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 59_2 60_16 69_67 70_24 71_192 79_67 80_24 81_192 89_2 90_16 91_4 102_99 103_24 112_99 113_24 122_99 123_24 152_2 179_198 188_8 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 35_49 36_140 37_70 38_48 45_49 46_140 47_2 48_49 49_132 55_33 56_128 57_2 58_49 59_140 67_2 68_57 69_204 77_3 78_24 79_198 87_3 88_24 89_192 95_68 98_24 99_192 104_8 105_198 106_32 114_8 115_198 116_32 124_8 125_198 126_32 152_1 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 37_1 38_8 47_49 48_140 49_64 57_49 58_140 59_64 60_64 67_33 68_136 102_96 112_99 122_99 132_97 135_70 136_33 137_8 138_64 142_3 143_24 144_192 145_198 146_49 147_140 148_98 152_3 153_24 154_192 155_70 156_49 157_140 158_98 163_24 164_198 165_6 166_33 167_140 168_66 174_198 175_48 185_49 186_128 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 70_16 71_128 79_3 80_24 81_192 89_3 90_24 91_196 100_24 101_192 103_1 104_16 113_35 114_24 115_128 123_35 124_24 125_128 132_8 133_35 134_24 135_128 143_66 146_4 147_97 153_198 156_12 157_99 163_6 166_12 167_99 176_4 177_97 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_8 37_64 53_2 54_16 57_2 58_16 63_99 64_16 65_128 67_67 68_24 69_192 73_99 74_16 75_128 77_67 78_24 79_198 80_48 83_35 84_16 87_3 88_24 89_198 90_49 91_128 99_198 100_49 101_128 103_1 104_16 109_6 110_57 111_196 113_35 114_24 115_128 119_2 120_56 121_196 123_35 124_24 125_128 130_16 131_128 133_35 134_24 135_128 140_24 141_192 149_1 150_24 151_192 158_1 159_9 160_88 161_192 168_1 169_140 170_248 178_1 179_140 180_128 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_8 37_71 38_24 47_2 48_16 54_134 55_32 64_142 65_113 74_134 75_49 76_140 77_64 82_1 83_8 84_4 85_33 86_140 87_96 92_35 93_24 94_64 95_33 96_140 97_96 102_35 103_28 104_192 105_1 106_8 112_1 113_8 114_64 136_6 146_198 147_48 154_8 155_66 156_198 157_48 164_8 165_103 166_54 167_48 174_8 175_102 185_64 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 38_142 48_142 49_113 50_128 58_7 59_49 60_176 68_3 69_56 70_165 71_128 78_3 79_24 80_199 81_32 89_16 90_198 91_49 98_8 99_66 100_70 101_49 107_1 108_28 109_226 110_6 111_33 117_1 118_8 119_226 128_8 129_64 137_4 138_99 147_132 148_99 149_16 157_132 158_99 159_16 167_4 168_32 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 53_12 62_1 63_136 70_16 71_192 80_24 81_192 91_198 101_16 110_70 111_49 120_70 121_49 128_8 129_198 130_70 131_49 138_24 139_230 140_16 141_132 148_74 149_198 150_49 151_128 157_140 158_103 159_114 160_177 161_128 167_140 168_231 169_57 170_148 177_140 178_231 179_57 180_128 186_1 187_140 188_102 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_71 37_57 38_196 46_71 47_57 48_220 49_198 56_67 57_24 58_217 59_206 60_48 66_3 67_156 68_89 69_206 70_49 71_128 76_17 77_140 78_98 79_198 80_49 81_128 86_1 87_140 88_99 89_30 90_177 91_128 98_99 99_24 100_128 104_66 105_16 106_128 108_35 109_24 110_132 114_71 115_56 116_128 119_16 120_198 121_49 124_66 125_16 126_128 130_70 131_49 135_2 136_16 140_70 141_49 145_66 146_49 147_136 155_66 156_49 157_136 165_2 166_49 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 44_70 45_16 53_24 54_199 55_48 56_128 62_3 63_24 64_198 65_48 72_3 73_24 74_66 82_99 83_8 92_97 102_96 104_70 105_16 107_1 108_140 109_32 112_32 114_70 115_48 116_128 117_17 118_140 119_96 124_70 125_48 126_128 127_1 128_140 129_33 130_8 134_2 135_16 138_4 139_33 140_140 141_96 145_24 146_194 149_33 150_140 151_96 154_1 155_24 156_198 159_5 160_237 161_64 165_24 166_198 169_12 170_96 175_8 176_192 179_12 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 54_3 55_24 56_192 64_3 65_24 66_192 74_3 75_24 76_192 82_132 83_98 85_16 92_140 93_99 102_132 103_99 112_4 113_32 122_8 123_66 128_1 129_24 130_196 132_140 133_226 138_1 139_24 140_196 142_4 143_226 144_16 148_1 149_24 150_196 153_66 158_67 159_57 160_140 165_132 166_33 167_8 168_231 169_57 170_140 175_206 176_99 177_28 178_231 179_57 180_128 185_78 186_99 187_24 188_198 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_132 44_2 45_17 46_196 47_32 54_70 55_49 56_206 57_32 63_8 64_199 65_48 66_132 73_8 74_198 75_16 82_99 83_24 84_194 88_8 89_99 90_8 91_64 92_99 93_156 94_226 97_1 98_12 99_99 100_28 101_64 102_99 103_156 104_192 107_1 108_12 109_99 110_28 111_194 112_99 113_24 114_64 118_8 119_99 120_8 121_67 122_96 131_3 132_32 138_140 139_32 147_17 148_140 149_96 153_1 154_8 157_17 158_140 159_32 163_33 164_28 165_192 168_132 173_1 174_28 175_192 185_64 186_24 187_198 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 37_49 38_140 47_49 48_140 49_96 57_1 58_140 59_8 60_68 68_33 69_8 70_198 71_32 74_2 77_4 78_35 79_24 80_198 81_32 83_8 84_99 85_16 87_4 88_99 89_24 90_192 93_8 94_99 95_24 97_4 98_35 99_16 104_99 105_16 108_8 109_66 118_8 119_226 125_2 126_49 127_8 128_12 129_230 130_32 135_70 136_51 137_156 138_202 139_206 140_113 145_70 146_51 147_156 148_227 149_206 150_115 155_70 156_51 157_156 158_227 159_142 160_113 165_70 166_49 167_140 168_64 169_142 170_96 175_2 176_49 177_136 179_4 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 35_24 36_198 46_198 47_4 48_99 56_16 57_140 58_99 65_2 66_49 67_140 68_99 72_1 73_136 75_66 76_49 77_140 78_32 82_49 83_136 84_66 85_18 86_49 87_128 92_49 93_136 94_71 95_24 96_128 104_71 105_56 106_128 113_1 114_90 115_148 117_2 123_35 124_25 125_206 126_32 127_71 128_48 129_128 133_35 134_25 135_206 136_121 137_231 138_56 139_128 143_35 144_25 145_206 146_121 147_231 148_56 149_128 153_35 154_24 155_196 156_32 157_199 158_48 163_35 164_24 165_128 167_2 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 73_4 76_132 77_33 82_16 83_198 84_48 85_49 86_142 87_113 92_24 93_198 94_54 95_49 96_142 97_113 102_24 103_198 104_54 105_49 106_140 107_96 112_24 113_231 114_62 115_177 116_140 123_99 124_24 125_199 126_177 133_35 134_24 135_198 136_49 137_136 143_1 144_24 145_198 146_49 147_140 148_35 149_16 154_99 155_18 156_49 157_4 158_99 159_24 163_12 164_99 165_24 167_4 168_99 169_24 174_99 175_24 178_32 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 76_6 77_49 78_128 86_198 87_49 88_128 96_198 97_49 98_140 105_17 106_206 107_49 108_140 113_2 114_49 115_149 116_206 117_49 118_140 120_66 121_16 123_7 124_57 125_140 126_132 127_37 128_161 130_99 131_24 133_7 134_57 135_206 136_48 137_134 138_115 139_8 140_99 141_24 143_2 144_57 145_199 146_57 147_134 148_115 150_66 154_16 155_199 156_57 157_132 158_160 165_2 166_16 167_49 168_140 177_49 178_140 179_32 186_2 187_49 188_140 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 49_198 56_8 57_99 58_24 59_198 60_48 65_1 66_140 67_99 68_24 69_198 70_49 71_128 75_33 76_140 77_99 78_24 85_33 86_140 87_99 88_24 95_1 96_12 97_96 118_1 119_8 120_196 128_1 129_24 130_198 131_32 138_1 139_8 140_198 160_2 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 77_49 78_128 86_6 87_49 88_140 96_6 97_49 98_140 99_96 106_6 107_115 108_140 109_96 114_17 115_140 116_36 117_115 118_140 119_96 120_2 121_16 124_57 125_204 126_32 127_37 128_173 129_8 130_67 131_24 134_57 135_206 136_113 137_132 138_51 139_152 140_67 141_24 144_25 145_206 146_57 147_204 148_51 149_152 150_2 155_134 156_57 157_204 158_37 166_16 167_129 168_140 169_96 177_17 178_140 179_96 187_17 188_140 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 114_2 115_48 116_128 124_6 125_57 126_128 134_70 135_49 136_128 142_1 143_24 144_198 145_16 149_67 150_24 151_192 152_3 153_28 154_231 155_16 157_33 158_8 159_103 160_24 161_224 163_28 164_231 165_16 166_4 167_51 168_140 169_99 170_24 174_66 176_4 177_35 178_136 179_66 187_33 188_8 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 82_198 92_230 93_16 102_198 103_16 105_8 107_33 108_8 112_199 113_48 114_1 115_140 116_70 117_115 118_136 122_199 123_48 124_161 125_156 126_102 127_115 128_136 132_66 133_16 134_33 135_142 136_80 137_161 143_4 144_49 145_203 146_57 147_204 153_6 154_113 155_235 156_57 157_206 163_4 164_53 165_99 166_57 167_198 174_4 175_99 176_24 177_198 185_99 186_24 187_198 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 103_198 104_48 105_17 106_128 112_24 113_198 114_54 115_49 116_140 117_32 122_24 123_198 124_54 125_49 126_206 127_113 132_24 133_198 134_54 135_49 136_206 137_113 143_231 144_24 145_241 146_140 147_97 153_35 154_24 155_198 156_49 157_136 163_35 164_24 165_198 166_49 167_140 168_33 174_67 175_2 176_49 177_140 178_99 179_24 185_24 187_4 188_98 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 49_66 58_8 59_71 60_16 67_33 68_24 69_231 70_16 77_99 78_24 79_194 80_16 87_99 88_24 89_64 95_1 96_132 97_33 98_8 101_4 105_49 106_140 107_32 110_16 111_134 114_2 115_49 116_140 117_32 119_2 120_57 121_198 124_70 125_49 126_132 129_2 130_57 131_196 133_24 134_199 135_48 136_128 139_2 140_16 141_128 142_3 143_24 144_231 145_48 151_64 152_3 153_28 154_231 155_16 161_192 163_24 164_230 165_16 167_4 168_32 174_227 175_24 177_134 178_96 185_24 187_134 188_98 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 77_1 78_8 87_33 88_140 89_64 97_33 98_140 99_64 107_1 108_8 121_198 126_132 129_2 130_16 131_198 134_2 135_49 136_206 137_32 139_67 140_24 141_198 144_2 145_57 146_206 147_48 149_99 150_24 151_192 154_2 155_57 156_206 157_32 159_67 160_24 161_128 164_66 165_16 166_132 174_231 175_16 185_16 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 82_192 92_194 95_16 96_196 97_32 102_194 104_35 105_153 106_206 107_32 112_230 113_4 114_51 115_152 116_196 117_32 122_194 123_4 124_49 125_138 133_134 134_57 135_103 136_56 143_206 144_61 145_103 146_57 147_128 153_135 154_188 155_103 156_57 157_192 163_5 164_140 165_99 166_24 167_192 174_12 175_99 176_24 177_192 185_1 186_24 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 38_12 47_1 48_140 49_99 57_33 58_140 59_98 67_49 68_140 69_64 76_4 77_49 78_140 81_128 87_49 88_136 90_24 91_198 97_49 98_140 100_24 101_198 106_4 107_49 108_140 110_16 111_198 116_4 117_49 118_140 126_4 127_33 136_198 137_32 145_16 146_206 147_48 150_64 155_25 156_206 157_48 159_12 160_99 164_2 165_57 166_206 167_48 169_12 170_96 174_2 175_57 176_206 177_49 178_128 179_8 185_16 186_198 187_49 188_140 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 38_8 47_1 48_28 49_227 57_35 58_156 59_227 67_35 68_156 69_227 77_35 78_24 79_66 86_4 87_115 88_136 96_140 97_115 98_136 105_17 106_140 107_97 110_2 115_49 116_140 117_97 120_67 121_24 125_17 126_140 127_49 128_140 130_99 131_56 136_4 137_49 138_140 140_67 141_16 147_49 148_140 157_1 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 38_8 48_8 49_231 58_8 59_98 67_35 68_156 69_98 77_99 78_156 79_226 87_99 88_156 89_98 91_6 97_33 98_8 99_64 115_16 116_132 117_32 125_49 126_206 127_96 130_66 134_2 135_49 136_206 137_113 139_8 140_99 141_16 145_49 146_142 147_49 148_128 149_8 150_103 156_134 157_49 158_140 159_8 160_98 166_6 167_49 168_128 " }},{ "match" : { "minutia.text": "0_7 1_224 2_31 3_248 4_63 5_252 6_127 7_254 8_127 9_254 10_255 11_255 12_255 13_255 14_255 15_255 16_255 17_255 18_255 19_255 20_255 21_255 22_127 23_254 24_127 25_254 26_63 27_252 28_31 29_248 30_7 31_224 36_4 37_49 38_140 46_4 47_51 48_140 49_96 56_4 57_49 58_140 66_4 67_49 68_136 76_6 77_49 78_136 86_6 87_49 88_140 90_3 91_24 96_6 97_49 98_140 100_3 101_24 106_6 107_49 108_140 109_96 110_2 111_24 116_4 117_49 118_140 119_99 128_140 129_99 138_4 139_97 " }}]}}}}}


In [ ]:
query = '''{
  "query": { 
    "bool": {
      "should":[{ 
          "bool":{
              "must": [ 
                { "match":  { "_id": "0" }},
                { "match": { "_index": "hash1"}}

              ]}},
          {"bool":
              {"must": [ 
                { "match":  { "_id": "0" }},
                { "match": { "_index": "hash0"}}
              ]
              }
          }
          
        ]
        
    }                                
  }, "size":0,
  "aggs":
      { "value_count" : {
          "terms":{
          "field" : "pos_text" }
          
      } }

  
    

}
'''

In [28]:
from textwrap import wrap

def generate_list_int(tuple_):
    line = tuple_[0]
    bits = tuple_[1]
    values_bit = wrap(line.replace(" ",""),bits)
    #print(len(values_bit))

    return list(map(lambda x1: int(x1,2),values_bit))



def generate_word(tuple_):
    text = tuple_[0]
    bits = tuple_[1]
    elem = generate_list_int((text,bits))
    text = ''
    #print(elem)
    list_not_found_8bits = [32, 33, 34, 39, 40, 41, 42, 51, 52, 61, 162, 171, 172, 181, 182,183, 184, 189, 190, 191, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 43, 50, 173, 180]

    for i,value in enumerate(elem):
        if(i not in list_not_found_8bits):

            text += '{}:{} '.format(i,value)
    
    
    
    #id = 1
    return text



In [29]:

import datetime  
from functools import reduce
from itertools import chain
import multiprocessing
import pandas as pd
import os
import numpy as np
es = Elasticsearch(hosts=ip)

#listNS = [18,16,12]
#listND = [5,5,7]
#list_quality = [0,15,20,30,40]
#list_bits = [8,12,16]


listNS = [16]
listND = [5]
list_quality = [0]
list_bits = [8]
pool_index = multiprocessing.Pool()
pool_insert = multiprocessing.Pool()
hashs = None
for ns,nd in zip(listNS,listND):
        
    for quality in list_quality:

       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}". format(ns,nd,quality))
       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
        print("    Vetores Gerados")

        for bits in list_bits:

            print("bits:", bits,"n_bits:",ns*ns*(nd+1),"hashs:",ns*ns*(nd+1)//bits, "quality:",quality )




            ###Indexação

            list_text = []

            for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
                for ind,file in enumerate(files):
                    if(True):#ind >=2000):
                         
                        if(ind == 500):
                            break


                        print(ind+1,file)
                        file_op = open(x+file)
                        vector_data = file_op.readlines() 
                        file_op.close()
                        list_text_min= pool_index.map(generate_word,zip(vector_data,list(np.repeat(bits,len(vector_data)))))
                        id = int(file.split('.')[0])
                        len_list = len(list_text_min)
                        list_text += list(gen_command(list_text_min,id))
                       
                        #print(hashs)
                        #print(len(hashs.text_in_hashs))
                        #list(map(generate_word, zip(list_positions, np.repeat(id,len_list), list(range(len_list)))))                          
                        
                            
                            
            print("    Bulk Genereted")
            a = helpers.bulk(es,list_text,request_timeout=1000)

            print("    Indexed")
            
            
        
    




    

    Vetores Gerados
bits: 8 n_bits: 1536 hashs: 192 quality: 0
1 1780.txt
2 1597.txt
3 1742.txt
4 2133.txt
5 1502.txt
6 1798.txt
7 1955.txt
8 1689.txt
9 2139.txt
10 1684.txt
11 1701.txt
12 1910.txt
13 4817.txt
14 2240.txt
15 1779.txt
16 1604.txt
17 1981.txt
18 1990.txt
19 1570.txt
20 1951.txt
21 2054.txt
22 2048.txt
23 1768.txt
24 1771.txt
25 1928.txt
26 1869.txt
27 1603.txt
28 1843.txt
29 1905.txt
30 1892.txt
31 4521.txt
32 1877.txt
33 2056.txt
34 1977.txt
35 1720.txt
36 1696.txt
37 2109.txt
38 4799.txt
39 4868.txt
40 2144.txt
41 1616.txt
42 1641.txt
43 1891.txt
44 1559.txt
45 1864.txt
46 4667.txt
47 5164.txt
48 4823.txt
49 4633.txt
50 1510.txt
51 2216.txt
52 4567.txt
53 1985.txt
54 1791.txt
55 1534.txt
56 1728.txt
57 4664.txt
58 1639.txt
59 5026.txt
60 5027.txt
61 2112.txt
62 1939.txt
63 1605.txt
64 1518.txt
65 2069.txt
66 1569.txt
67 1602.txt
68 2055.txt
69 1972.txt
70 1971.txt
71 1811.txt
72 1761.txt
73 4866.txt
74 1576.txt
75 1880.txt
76 2223.txt
77 1974.txt
78 2071.txt
79 1589.tx

In [60]:
list_text[1]

{'_op_type': 'create',
 '_index': 'finger1',
 '_source': {'id': 1780,
  'min': 1,
  'text': '0:7 1:224 2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 12:255 13:255 14:255 15:255 16:255 17:255 18:255 19:255 20:255 21:255 22:127 23:254 24:127 25:254 26:63 27:252 28:31 29:248 30:7 31:224 32:0 33:0 34:0 35:68 36:0 37:0 38:0 39:0 40:0 41:0 42:0 43:1 44:24 45:196 46:32 47:0 48:0 49:70 50:0 51:0 52:0 53:35 54:24 55:196 56:0 57:0 58:0 59:70 60:48 61:0 62:0 63:35 64:24 65:132 66:0 67:0 68:0 69:0 70:0 71:32 72:0 73:1 74:16 75:128 76:0 77:0 78:0 79:16 80:0 81:0 82:0 83:0 84:0 85:0 86:0 87:0 88:35 89:24 90:128 91:0 92:0 93:0 94:0 95:0 96:0 97:0 98:35 99:24 100:128 101:0 102:0 103:0 104:0 105:0 106:0 107:0 108:35 109:24 110:128 111:0 112:0 113:0 114:0 115:0 116:0 117:0 118:3 119:24 120:132 121:0 122:0 123:0 124:0 125:0 126:0 127:0 128:35 129:24 130:196 131:0 132:0 133:0 134:0 135:0 136:0 137:0 138:3 139:24 140:132 141:0 142:0 143:0 144:0 145:0 146:0 147:0 148:0 149:6 150:48 151:128 152

In [14]:

def gendata1(list_text):

  #  list_not_index= []

    for i,pos in enumerate(list_text):
        yield {"index": "finger1" }

        yield {"_source":["id"],"size":100,"query": {"match":{"text": pos}} }

                            
                            
                            
                            

In [71]:
s = {,
  "query": {
    "query_string": {
      "query": "0:7",
      "default_field": "field"
    
    }
  }
}

In [40]:
s = {"_source":["id"], "size":100,
   "query": {
    "match":{
     "text":  list_text_min[0]  
    }
  }
}


In [15]:
list(gendata1([ "2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 ","2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 ", "2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 "]))

[{'index': 'finger1'},
 {'_source': ['id'],
  'size': 100,
  'query': {'match': {'text': '2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 '}}},
 {'index': 'finger1'},
 {'_source': ['id'],
  'size': 100,
  'query': {'match': {'text': '2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 '}}},
 {'index': 'finger1'},
 {'_source': ['id'],
  'size': 100,
  'query': {'match': {'text': '2:31 3:248 4:63 5:252 6:127 7:254 8:127 9:254 10:255 11:255 '}}}]

In [47]:
%%time
data_response = es.msearch(body=list(gendata1(list_text_min)),request_timeout=1000)


CPU times: user 2.5 s, sys: 5.07 s, total: 7.57 s
Wall time: 55.4 s


In [46]:
%%time
a = es.search(body=s,index='finger1')

CPU times: user 59 ms, sys: 159 ms, total: 219 ms
Wall time: 2.23 s


In [20]:
a

{'took': 423,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 37.930954,
  'hits': [{'_index': 'finger1',
    '_type': '_doc',
    '_id': '5UjgsHIBTAOKEQx95Ze8',
    '_score': 37.930954,
    '_source': {'id': 1543}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'gEnhsHIBTAOKEQx9uxp3',
    '_score': 37.65575,
    '_source': {'id': 1651}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'b0nisHIBTAOKEQx9w7Cr',
    '_score': 37.253746,
    '_source': {'id': 4973}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'vEnisHIBTAOKEQx9IVf7',
    '_score': 37.128063,
    '_source': {'id': 1517}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'U0nisHIBTAOKEQx9wK9p',
    '_score': 37.113815,
    '_source': {'id': 2143}},
   {'_index': 'finger1',
    '_type': '_doc',
    '_id': 'e0jgsHIBTAOKEQx9t3v2',
    '_score': 37.073177,
    '_source': {'id'

In [54]:
a = '''{
  "query": { 
    "bool": {
      "should":[{ 
          "bool":{
              "must": [ 
                { "match":  { "_id": "0" }},
                { "match": { "_index": "hash1"}}

              ]}},
          {"bool":
              {"must": [ 
                { "match":  { "_id": "0" }},
                { "match": { "_index": "hash0"}}
              ]
              }
          }
          
        ]
        
    }                                
  }, "size":0,
  "aggs":
      { "value_count" : {
          "terms":{
          "field" : "pos_text" }
          
      } }

  
    

}
'''

In [38]:

import datetime  
from functools import reduce
from itertools import chain
import multiprocessing
import pandas as pd
import os
import numpy as np
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch(hosts=ip)

#listNS = [18,16,12]
#listND = [5,5,7]
#list_quality = [0,15,20,30,40]
#list_bits = [8,12,16]


listNS = [16]
listND = [5]
list_quality = [15]
list_bits = [8]
pool_index = multiprocessing.Pool()
pool_insert = multiprocessing.Pool()
hashs = None
for ns,nd in zip(listNS,listND):
        
    for quality in list_quality:

      #  os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}". format(ns,nd,quality))
       # os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
        print("    Vetores Gerados")

        for bits in list_bits:

            print("bits:", bits,"n_bits:",ns*ns*(nd+1),"hashs:",ns*ns*(nd+1)//bits, "quality:",quality )
  
            ###Search

            pool = multiprocessing.Pool()


            pos_rank =[]
            for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Search/"):
                count = 0
                for file in files:

                    if(True):#count >= 618):

                        file_op = open(x+file)
                        vector_data = file_op.readlines() 
                        file_op.close()
                        time1 = datetime.datetime.now()
                        list_text_min= pool.map(generate_word,zip(vector_data,list(np.repeat(bits,len(vector_data)))))
                        #id = int(file.split('.')[0])
                        #len_list = len(list_text_min)
                        #list_text += list(gen_command(list_text_min,id))
                        print(len(list_text_min)) 
                        data_response = es.msearch(body=list(gendata1(list_text_min)),request_timeout=1000)

                        #a = pool.map(realize_rank,zip(vector_data,np.repeat(bits,len(vector_data)),np.repeat([index_not_found],len(vector_data),axis=0)))
                        
                        #a = map(realize_rank,zip(vector_data,np.repeat(bits,len(vector_data))))

                       # persons = list(chain.from_iterable(a))

                        #ranking = func_rank_elements(persons)
                        time2 = datetime.datetime.now()
                        id = file.split(".")[0]
                        pos_  = -1
                        #for p in range(len(ranking)):
                         #   if(id  == ranking[p]):
                          #      pos_ = p + 1
                           #     break
                        print("count:",count,"filename:",file,"pos_rank:",pos_,"time:",time2-time1)
                        pos_rank.append([pos_,file,time2-time1])
                        
                    count+=1
                    break
                    
            

            df = pd.DataFrame(pos_rank,columns=['pos_rank','file','time'])
           # df.to_csv('Resultado_Preliminar/result_query{}_{}_{}.csv'.format(ns*ns*(nd+1),quality,bits),index=False)


    Vetores Gerados
bits: 8 n_bits: 1536 hashs: 192 quality: 15
126
count: 0 filename: 368.txt pos_rank: -1 time: 0:00:53.855753


'35:0 36:0 37:0 38:0 44:0 45:0 46:0 47:0 48:0 49:0 53:0 54:0 55:0 56:0 57:0 58:0 59:0 60:0 62:0 63:0 64:0 65:0 66:0 67:0 68:0 69:0 70:0 71:0 72:0 73:0 74:0 75:0 76:0 77:0 78:0 79:0 80:0 81:0 82:0 83:0 84:0 85:0 86:0 87:0 88:0 89:0 90:0 91:0 92:0 93:0 94:0 95:0 96:0 97:0 98:0 99:0 100:0 101:0 102:0 103:0 104:0 105:0 106:0 107:0 108:0 109:0 110:0 111:0 112:0 113:0 114:0 115:0 116:0 117:0 118:0 119:0 120:0 121:0 122:0 123:0 124:0 125:0 126:0 127:0 128:0 129:0 130:0 131:0 132:0 133:0 134:0 135:0 136:0 137:0 138:8 139:64 140:0 141:0 142:0 143:0 144:0 145:0 146:0 147:1 148:12 149:98 150:0 151:0 152:0 153:0 154:0 155:0 156:0 157:1 158:12 159:99 160:0 161:0 163:0 164:0 165:0 166:0 167:0 168:8 169:98 170:0 174:0 175:0 176:0 177:0 178:0 179:0 185:0 186:0 187:0 188:0 '

In [ ]:
%%time
import numpy as np
from textwrap import wrap
import os

ns = 16
nd = 5
quality = 0

os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}".format(ns,nd,quality))
os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
       
print("Generate Vectors")
    
    
es = Elasticsearch(hosts=ip)

for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
    for ind,file in enumerate(files):
        if(True):#ind >=2000):
            if(ind %300 == 0 and ind != 0):
                list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
                print(len(list_for_bulk))
                tam = len(list_for_bulk)
                #l1 = list_for_bulk[0:10]
                #l2 = list_for_bulk[int(tam/2):tam]
                helpers.bulk(es,list_for_bulk,request_timeout=1000)
                print("Send bulk")

                hashs = Hash(96,16)
                print("Clear Hash")

              #  break

            print(ind+1,file)
            vector_data = sc.textFile(x + file)
            list_positions = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,16)).map(lambda x: list(map(lambda x1: int(x1,2),x)) ).collect()
            id = int(file.split('.')[0])
            len_list = len(list_positions)
            print(len_list)
            list(map(generate_word, list_positions, np.repeat(id,len_list), list(range(len_list))))

            

list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
print(len(list_for_bulk))
tam = len(list_for_bulk)
#l1 = list_for_bulk[0:10]
#l2 = list_for_bulk[int(tam/2):tam]
helpers.bulk(es,list_for_bulk,request_timeout=1000)
print("Send bulk")

hashs = Hash(64,24)
print("Clear Hash")



In [72]:
es = Elasticsearch(hosts=ip)

helpers.bulk(es,list_for_bulk,request_timeout=1000)


(4, [])

In [5]:
def set_value_in_hash(string_text,index,position):
    global hashs
    
    hashs.text_in_hashs[index][position]+=string_text
    
def generate_word(list_index,id,id_min):
    #id = 1
    return list(map(lambda pos_hash,index_hash: set_value_in_hash("{}_{} ".format(id,id_min),index_hash,pos_hash) if(pos_hash > 0) else None ,list_index,list(range(len(list_index)))))

In [22]:
import os

ns = 16
nd = 5
quality = 0

os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}".format(ns,nd,quality))
os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
       
print("Generate Vectors")

Generate Vectors


In [ ]:
%%time
import numpy as np
from textwrap import wrap
import os

ns = 16
nd = 5
quality = 20

#os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_index.txt  ../data-fingerprint/SD14/Finger_data/Vector_Index/ {} {} {}".format(ns,nd,quality))
#os.system("./../source/generate_vector_python /home/johnny/Projeto/TCC/data-fingerprint/SD14/file_min_search.txt ../data-fingerprint/SD14/Finger_data/Vector_Search/ {} {} {}".format(ns,nd,quality))
       
print("Generate Vectors")
    
    
es = Elasticsearch(hosts=ip)
hashs = Hash(96,16)
print("Create hash")
hashs_boolean = Hash_Boolean(96,16)
print("Create hash Boolean")

for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Index/"):
    for ind,file in enumerate(files):
        if(True):#ind >=2000):
            if(ind %300 == 0 and ind != 0):
                list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
                print(len(list_for_bulk))
                tam = len(list_for_bulk)
                #l1 = list_for_bulk[0:10]
                #l2 = list_for_bulk[int(tam/2):tam]
                helpers.bulk(es,list_for_bulk,request_timeout=1000)
                print("Send bulk")

                hashs = Hash(96,16)
                print("Clear Hash")

              #  break

            print(ind+1,file)
            vector_data = sc.textFile(x + file)
            list_positions = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,16)).map(lambda x: list(map(lambda x1: int(x1,2),x)) ).collect()
            id = int(file.split('.')[0])
            len_list = len(list_positions)
            print(len_list)
            list(map(generate_word, list_positions, np.repeat(id,len_list), list(range(len_list))))

            

list_for_bulk = convert_hash_to_bulk(hashs,hashs_boolean)
print(len(list_for_bulk))
tam = len(list_for_bulk)
#l1 = list_for_bulk[0:10]
#l2 = list_for_bulk[int(tam/2):tam]
helpers.bulk(es,list_for_bulk,request_timeout=1000)
print("Send bulk")

hashs = Hash(64,24)
print("Clear Hash")



Generate Vectors
Create hash
Create hash Boolean
1 1597.txt
200
2 2133.txt
89
3 1502.txt
124
4 1798.txt
100
5 2139.txt
144
6 1701.txt
140
7 1604.txt
102
8 1981.txt
123
9 1990.txt
92
10 2054.txt
110
11 2048.txt
126
12 1768.txt
87
13 1928.txt
161
14 1869.txt
78
15 1603.txt
100
16 1843.txt
90
17 2056.txt
143
18 1977.txt
147
19 1720.txt
61
20 1696.txt
118
21 1616.txt
119
22 1559.txt
91
23 1864.txt
96
24 1510.txt
84
25 1985.txt
89
26 1791.txt
101
27 1534.txt
173
28 1639.txt
137
29 2112.txt
172
30 1939.txt
134
31 1605.txt
77
32 1518.txt
115
33 2069.txt
115
34 1602.txt
74
35 2055.txt
128
36 1972.txt
20
37 1971.txt
171
38 1761.txt
141
39 1576.txt
130
40 2223.txt
107
41 1974.txt
133
42 2071.txt
229
43 1589.txt
97
44 1627.txt
103
45 1630.txt
99
46 1845.txt
68
47 1577.txt
112
48 1695.txt
49
49 1788.txt
145
50 1927.txt
113
51 1793.txt
92
52 1943.txt
125
53 2157.txt
190
54 1594.txt
173
55 2046.txt
142
56 1693.txt
123
57 1805.txt
100
58 1596.txt
207
59 1764.txt
76
60 1961.txt
79
61 1832.txt
133
62 1

In [16]:
import numpy as np
from textwrap import wrap
import os

In [17]:

def gendata1(list_pos_in_hash):
   # list_not_index = [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
    #    13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
    #    26,  27,  28,  29,  30,  31,  43,  50, 173, 180, 32,  33,  34,  39,  40,  41,  42,  51,  52,  61, 162, 171, 172,
    #   181, 182, 183, 184, 189, 190, 191]
    #list_not_index=[0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 21,
     #               25, 86, 90,16, 20, 91, 95]
        
    list_not_index = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 21,25,16,20]
  #  list_not_index= []
    for i,pos in enumerate(list_pos_in_hash):
        if(pos != 0 and i not in list_not_index):
            yield {"index": "hash{}".format(i) }
            yield {
                    "query": {"terms": {"_id":[str(pos)]}}} 
                            
                            
                            
                            
                            
                            
      

In [18]:
def realize_search(data_consult):
    global ip
    from elasticsearch import Elasticsearch
    es = Elasticsearch(hosts=ip)
    if(data_consult != []):
        data_response = es.msearch(body=data_consult)
       # print(data_response)
      #  print(data_response)
      #  return data_response
      #  return []
        return list(map(lambda x : x['hits']['hits'][0]['_source']['text'] if x['hits']['hits'] != [] else '', data_response['responses']))
    else:
        return []
    

In [19]:

import numpy as np

def func_count_byKey_return_max(x):
    if(x != []):
        import numpy as np
        x_,y_ = np.unique(x,return_counts=True)
        y_max = np.max(y_)
        return " ".join([ i[0].split('_')[0] for i in list(filter(lambda x: x[1] == y_max,list(map(lambda x,y:(x,y) ,x_,y_)))) ])
    else:
        return ""
def func_rank_elements(persons):
    persons_list, count_ = np.unique(persons.split(),return_counts=True)
    return persons_list[np.argsort(-1*count_)]


In [21]:
import datetime                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
pos_rank =[]
for x,i,files in os.walk("../data-fingerprint/SD14/Finger_data/Vector_Search/"):
    count = 0
    for file in files:
        
       # print(file)
        if(True):#count >= 618):
            time1 = datetime.datetime.now()
            vector_data = sc.textFile(x + file)
            persons = vector_data.map(lambda x:x.replace(" ","")).map(lambda x:wrap(x,16)).map(lambda x: list(map(lambda x1: int(x1,2),x))).map(lambda x : list(gendata1(x))).map(lambda x: realize_search(x)).map(lambda x : " ".join(x)).map(lambda x : x.split()).map(func_count_byKey_return_max ).reduce(lambda x,y: y + " "+ x).replace("  ", " ")
            ranking = func_rank_elements(persons)
            time2 = datetime.datetime.now()
            id = file.split(".")[0]
            pos_  = -1
            for p in range(len(ranking)):
                #print(ranking[p] ," a ")
                if(id  == ranking[p]):
                    pos_ = p + 1
           #         break
            print("count:",count,"filename:",file,"pos_rank:",pos_,"time:",time2-time1)
            pos_rank.append([pos_,file,time2-time1])
 #       break
        count+=1

        

count: 0 filename: 1597.txt pos_rank: 1 time: 0:01:06.643205


KeyboardInterrupt: 

In [41]:
import pandas as pd
df = pd.DataFrame(pos_rank,columns=['rank_pos','file','time'])
df.to_csv('result_new_method_secretaria.csv',index=False)

In [42]:
df[(df['rank_pos'] > 0) &(df['rank_pos'] <=52 )].shape

(856, 3)

In [43]:
1 - 848/1020

0.1686274509803921

In [44]:
1 - 438/1020

0.5705882352941176

In [46]:
1 - 856/1020

0.16078431372549018

In [3]:
from elasticsearch import Elasticsearch
import  pandas as pd
es = Elasticsearch(hosts=ip)
count_index_voids = []
for i in range(64):
    doc = {
        'size' : 10000,
        'query': {
            'match_all' : {}
       }
    }
    data_r = es.search(index='hash{}'.format(i), doc_type='pos', body=doc,scroll='1m')
    #data_r = es.get(index="hashs",id=i,doc_type="hash")
    count_void_elements = 0
    
    for partial_data in data_r['hits']['hits']:
            #if(partial_data['_source']['text'] == ''):
            count_void_elements +=1
      #      else:
       #         print(partial_data['_source']['text'])
        
    print(count_void_elements)
    scrollId = data_r['_scroll_id']
    scrollId_ant = ''
    while scrollId != scrollId_ant:
        print(scrollId, count_void_elements)
        data_r = es.scroll(scroll_id = scrollId, scroll = '1m')
        for partial_data in data_r['hits']['hits']:
            #if(partial_data['_source']['text'] == ''):
            count_void_elements +=1
            
        scrollId_ant = scrollId
        scrollId = data_r['_scroll_id']

         #   else:
        #        print(partial_data['_id'],partial_data['_index'])
                #print(partial_data['_source']['text'])
        
     #  ' if(count_void_elements len(data_r['_source']['index_in_hash']))
    count_index_voids.append([count_void_elements,i])
    print(i)
  #  break

/home/johnny/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAABFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAAhZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAAMWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAAEFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAABRZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
0
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAAGFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAChZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAAcWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAAIFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAACRZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
1
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAALFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAADBZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAA0WVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAAOFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAADxZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
2
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAAQFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAERZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAABMWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAASFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAFBZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1
3
1
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAAVFlZJSTZFTDdQUl82NEExeklBSk

34
575
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAACwFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAshZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAALEWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAACzFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAtBZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 575
35
596
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAC1FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAthZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAALcWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAC4FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAuRZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 596
36
1795
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAC6FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAuxZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAALwWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAAC9FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAvhZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1795
37
1803
DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAC_FlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAwhZWSUk2RUw3UFJfNjRBMXpJQUpPaHdBAAAAAAAAAMAWVklJNkVMN1BSXzY0QTF6SUFKT2h3QQAAAAAAAADBFlZJSTZFTDdQUl82NEExeklBSk9od0EAAAAAAAAAwxZWSUk2RUw3UFJfNjRBMXpJQUpPaHdB 1803
38
1866
DnF1ZXJ5VGhlbkZldGNoBQAAAAAA

In [9]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame(count_index_voids,columns=['voids','index'])


In [6]:
df

,voids,index
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
59,1648,59
60,1876,60
61,1867,61
62,1822,62


In [7]:
df.describe()

,voids,index
count,64.000000,64.000000
mean,1050.328125,31.500000
std,864.959528,18.618987
min,0.000000,0.000000
25%,1.000000,15.750000
50%,1635.500000,31.500000
75%,1833.000000,47.250000
max,2018.000000,63.000000


In [10]:
np.unique(df.iloc[:,0],return_counts=True)

(array([   0,    1,   60,   62,  112,  117,  575,  587,  596,  619, 1340,
        1439, 1453, 1627, 1644, 1648, 1655, 1703, 1707, 1734, 1751, 1757,
        1768, 1795, 1803, 1809, 1818, 1822, 1825, 1831, 1839, 1852, 1866,
        1867, 1870, 1876, 1888, 1923, 1927, 1944, 1951, 1959, 1976, 2018]),
 array([ 2, 18,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,
         1,  1,  1,  2,  1,  1,  1,  1,  1,  1]))

In [11]:
df.median()

voids    1635.5
index      31.5
dtype: float64

### Existem 20 hashs em que todos os elementos caíram em 0 e foram eliminados

In [12]:
df[df['voids'] ==0].shape

(2, 2)

In [13]:
df[df['voids'] ==0]['index'].values

array([16, 20])

### Existem 36 hashs que todos os elementos estão na mesma posição, ou seja, são dados agrupados na mesma posição (muito volume) e são irrelevantes

In [14]:
df[df['voids'] ==1].shape

(18, 2)

In [15]:
df[df['voids'] ==1]['index'].values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 21,
       25])